<a href="https://colab.research.google.com/github/ramayer/google-colab-examples/blob/main/Efficient_spark_range_joins_in_Databricks_vs_Apache_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Efficient spark range joins in Databricks vs Apache Spark

## Databricks spark has efficient range joins through hints.

https://docs.databricks.com/en/optimizations/range-join.html


## Apache Spark seems not to, yet.

* https://github.com/apache/spark/pull/7379
* https://issues.apache.org/jira/browse/SPARK-8682

Zach Moshe describes a workaround.

* http://zachmoshe.com/2016/09/26/efficient-range-joins-with-spark.html

This notebook implements something simlar to each of the above.


In [5]:
try:
  import pyspark, findspark, delta
except:
   %pip install -q --upgrade pyspark==3.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
import pyspark

MAX_MEMORY="8g"
maven_coords = [
    'io.delta:delta-spark_2.12:3.2.0',
]
spark = (pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.jars.packages", ",".join(maven_coords))
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.executor.memory", MAX_MEMORY)
    .config("spark.driver.memory", MAX_MEMORY)
    .enableHiveSupport()
    .getOrCreate()
    )
spark

In [107]:
spark.range(1000 * 1000).createOrReplaceTempView("a_million_rows")

In [108]:
df1 = spark.sql("""
  select
    cast('2024-01-01' as timestamp) + interval '1 second' * id /4 as dttm,
    sin(id/60 / 4) as sin,
    cos(id/60 / 4) as cos
  from a_million_rows
  """)
df1.write.format("delta").mode('overwrite').saveAsTable("df1")
df1.sort('dttm').limit(3).pandas_api()

,dttm,sin,cos
0,2024-01-01 00:00:00.000,0.000000,1.000000
1,2024-01-01 00:00:00.250,0.004167,0.999991
2,2024-01-01 00:00:00.500,0.008333,0.999965


In [109]:
df2 = spark.sql("""
  select
    cast('2024-01-01' as timestamp) + interval '1 second' * id as dttm,
    cos(id*1.5/60) as val
  from a_million_rows
  """)
df2.write.format("delta").mode('overwrite').saveAsTable("df2")
df2.sort('dttm').limit(3).pandas_api()

,dttm,val
0,2024-01-01 00:00:00,1.000000
1,2024-01-01 00:00:01,0.999688
2,2024-01-01 00:00:02,0.998750


In [110]:
df1 = spark.table("df1")
df2 = spark.table("df2")

In [111]:
from pyspark.sql.functions import current_timestamp, lag
from pyspark.sql.window import Window
df2_ranges = (spark.sql("select * from df2")
                .withColumn("prev_dttm", lag("dttm").over(Window.orderBy("dttm")))
                .withColumn("prev_val", lag("val").over(Window.orderBy("dttm")))
                .selectExpr("prev_dttm","prev_val", "dttm as next_dttm", "val as next_val")
                )
df2_ranges.createOrReplaceTempView("df2_ranges")
df2_ranges.sort('prev_dttm').limit(3).pandas_api()

,prev_dttm,prev_val,next_dttm,next_val
0,NaT,NaN,2024-01-01 00:00:00,1.000000
1,2024-01-01 00:00:00,1.000000,2024-01-01 00:00:01,0.999688
2,2024-01-01 00:00:01,0.999688,2024-01-01 00:00:02,0.998750


In [112]:
# Don't run this on F/OSS Apache Spark, it takes forever
# CartesianProduct and/or BroadcastNestedLoopJoin
# (depending on spark configs) are both painful ways to do joins.

spark.sql("""
  SELECT df1.dttm as df1_dttm,
         df1.sin  as df1_sin,
         df1.cos  as df1_cos,
         df2_ranges.prev_dttm,
         df2_ranges.next_dttm,
         df2_ranges.prev_val,
         df2_ranges.next_val
  FROM df1
  JOIN df2_ranges ON (df1.dttm >= df2_ranges.prev_dttm
                  AND df1.dttm <= df2_ranges.next_dttm)
""").explain()



== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [dttm#33021 AS df1_dttm#33018, sin#33022 AS df1_sin#33019, cos#33023 AS df1_cos#33020, prev_dttm#32119, next_dttm#32128, prev_val#32123, next_val#32129]
   +- CartesianProduct ((dttm#33021 >= prev_dttm#32119) AND (dttm#33021 <= next_dttm#32128))
      :- Filter isnotnull(dttm#33021)
      :  +- FileScan parquet spark_catalog.default.df1[dttm#33021,sin#33022,cos#33023] Batched: true, DataFilters: [isnotnull(dttm#33021)], Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[file:/content/spark-warehouse/df1], PartitionFilters: [], PushedFilters: [IsNotNull(dttm)], ReadSchema: struct<dttm:timestamp,sin:double,cos:double>
      +- Project [prev_dttm#32119, prev_val#32123, dttm#32114 AS next_dttm#32128, val#32115 AS next_val#32129]
         +- Filter (isnotnull(prev_dttm#32119) AND isnotnull(dttm#32114))
            +- Window [lag(dttm#32114, -1, null) windowspecdefinition(dttm#32114 ASC NULLS FIRST, specifiedwindowfra

In [113]:
# python API does no better, even with a hint
df1.hint("range_join",6).join(df2_ranges,
                               on=[
                                   df1.dttm >= df2_ranges.prev_dttm,
                                   df1.dttm <= df2_ranges.next_dttm
                               ]).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- CartesianProduct ((dttm#32104 >= prev_dttm#32119) AND (dttm#32104 <= next_dttm#32128))
   :- Filter isnotnull(dttm#32104)
   :  +- FileScan parquet spark_catalog.default.df1[dttm#32104,sin#32105,cos#32106] Batched: true, DataFilters: [isnotnull(dttm#32104)], Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[file:/content/spark-warehouse/df1], PartitionFilters: [], PushedFilters: [IsNotNull(dttm)], ReadSchema: struct<dttm:timestamp,sin:double,cos:double>
   +- Project [prev_dttm#32119, prev_val#32123, dttm#32114 AS next_dttm#32128, val#32115 AS next_val#32129]
      +- Filter (isnotnull(prev_dttm#32119) AND isnotnull(dttm#32114))
         +- Window [lag(dttm#32114, -1, null) windowspecdefinition(dttm#32114 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, -1)) AS prev_dttm#32119, lag(val#32115, -1, null) windowspecdefinition(dttm#32114 ASC NULLS FIRST, specifiedwindowframe(RowFrame, -1, -1)) AS prev_val#32123], [dttm#

In [114]:
# This should be fast on Databricks
# https://docs.databricks.com/en/optimizations/range-join.html
# but is annoyingly not on Apache Spark
spark.sql("""
  SELECT  /*+ RANGE_JOIN(dttm, 10) */
         df1.dttm as df1_dttm,
         df1.sin  as df1_sin,
         df1.cos  as df1_cos,
         df2_ranges.prev_dttm,
         df2_ranges.next_dttm,
         df2_ranges.prev_val,
         df2_ranges.next_val
  FROM df1
  JOIN df2_ranges ON (df1.dttm >= df2_ranges.prev_dttm
                  AND df1.dttm <= df2_ranges.next_dttm)
""").explain()



== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [dttm#33704 AS df1_dttm#33701, sin#33705 AS df1_sin#33702, cos#33706 AS df1_cos#33703, prev_dttm#32119, next_dttm#32128, prev_val#32123, next_val#32129]
   +- CartesianProduct ((dttm#33704 >= prev_dttm#32119) AND (dttm#33704 <= next_dttm#32128))
      :- Filter isnotnull(dttm#33704)
      :  +- FileScan parquet spark_catalog.default.df1[dttm#33704,sin#33705,cos#33706] Batched: true, DataFilters: [isnotnull(dttm#33704)], Format: Parquet, Location: PreparedDeltaFileIndex(1 paths)[file:/content/spark-warehouse/df1], PartitionFilters: [], PushedFilters: [IsNotNull(dttm)], ReadSchema: struct<dttm:timestamp,sin:double,cos:double>
      +- Project [prev_dttm#32119, prev_val#32123, dttm#32114 AS next_dttm#32128, val#32115 AS next_val#32129]
         +- Filter (isnotnull(prev_dttm#32119) AND isnotnull(dttm#32114))
            +- Window [lag(dttm#32114, -1, null) windowspecdefinition(dttm#32114 ASC NULLS FIRST, specifiedwindowfra

## Manually emulate the Databricks optimization

In [115]:
# Choose a minute for the bin size.
# Smaller bins are faster, but less forgiving of missing data.
spark.sql("""
  create or replace temp view df1b as
  select *,
    floor(unix_timestamp(dttm)/60) as bin
  from df1
""")
spark.sql("""
  create or replace temp view df2b as
  select *,
    floor(unix_timestamp(prev_dttm)/60) as prev_bin,
    floor(unix_timestamp(next_dttm)/60) as next_bin
  from df2_ranges
""")


DataFrame[]

In [116]:
# Much better plan.
#
# SortMergeJoin & hashpartitioning should be reasonable.

manually_binned_join = spark.sql("""
WITH a as (
   SELECT df1b.dttm as df1_dttm,
         df1b.sin  as df1_sin,
         df1b.cos  as df1_cos,
         df2b.prev_dttm,
         df2b.next_dttm,
         df2b.prev_val,
         df2b.next_val
   FROM df1b
   JOIN df2b ON (df1b.bin = df2b.next_bin)
  ),
  b as (
  SELECT df1b.dttm as df1_dttm,
         df1b.sin  as df1_sin,
         df1b.cos  as df1_cos,
         df2b.prev_dttm,
         df2b.next_dttm,
         df2b.prev_val,
         df2b.next_val
  FROM df1b
  JOIN df2b ON (df1b.bin <> df2b.next_bin and df1b.bin = df2b.prev_bin)
  ),
  c as (
    SELECT * FROM a
    UNION ALL
    SELECT * FROM b
  )
  SELECT * FROM c
  where (df1_dttm >= prev_dttm and df1_dttm < next_dttm)
""")
manually_binned_join.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Union
   :- Project [dttm#33992 AS df1_dttm#33965, sin#33993 AS df1_sin#33966, cos#33994 AS df1_cos#33967, prev_dttm#32119 AS prev_dttm#33968, next_dttm#33941 AS next_dttm#33969, prev_val#32123 AS prev_val#33970, next_val#33942 AS next_val#33971]
   :  +- SortMergeJoin [bin#33932L], [next_bin#33944L], Inner, ((dttm#33992 >= prev_dttm#32119) AND (dttm#33992 < next_dttm#33941))
   :     :- Sort [bin#33932L ASC NULLS FIRST], false, 0
   :     :  +- Exchange hashpartitioning(bin#33932L, 200), ENSURE_REQUIREMENTS, [plan_id=19557]
   :     :     +- Project [dttm#33992, sin#33993, cos#33994, FLOOR((cast(unix_timestamp(dttm#33992, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC), false) as double) / 60.0)) AS bin#33932L]
   :     :        +- Filter (isnotnull(dttm#33992) AND isnotnull(FLOOR((cast(unix_timestamp(dttm#33992, yyyy-MM-dd HH:mm:ss, Some(Etc/UTC), false) as double) / 60.0))))
   :     :           +- FileScan parquet spark_catalog.default.

In [117]:
import time
import pandas
t0 = time.time()
manually_binned_join.createOrReplaceTempView("manually_binned_join")
result = spark.sql("select * from manually_binned_join order by df1_dttm").limit(10).toPandas()
t1 = time.time()
print(f"took {t1-t0:.2f} seconds")
result

took 6.50 seconds


,df1_dttm,df1_sin,df1_cos,prev_dttm,next_dttm,prev_val,next_val
0,2024-01-01 00:00:00.000,0.000000,1.000000,2024-01-01 00:00:00,2024-01-01 00:00:01,1.000000,0.999688
1,2024-01-01 00:00:00.250,0.004167,0.999991,2024-01-01 00:00:00,2024-01-01 00:00:01,1.000000,0.999688
2,2024-01-01 00:00:00.500,0.008333,0.999965,2024-01-01 00:00:00,2024-01-01 00:00:01,1.000000,0.999688
3,2024-01-01 00:00:00.750,0.012500,0.999922,2024-01-01 00:00:00,2024-01-01 00:00:01,1.000000,0.999688
4,2024-01-01 00:00:01.000,0.016666,0.999861,2024-01-01 00:00:01,2024-01-01 00:00:02,0.999688,0.998750
5,2024-01-01 00:00:01.250,0.020832,0.999783,2024-01-01 00:00:01,2024-01-01 00:00:02,0.999688,0.998750
6,2024-01-01 00:00:01.500,0.024997,0.999688,2024-01-01 00:00:01,2024-01-01 00:00:02,0.999688,0.998750
7,2024-01-01 00:00:01.750,0.029163,0.999575,2024-01-01 00:00:01,2024-01-01 00:00:02,0.999688,0.998750
8,2024-01-01 00:00:02.000,0.033327,0.999444,2024-01-01 00:00:02,2024-01-01 00:00:03,0.998750,0.997189
9,2024-01-01 00:00:02.250,0.037491,0.999297,2024-01-01 00:00:02,2024-01-01 00:00:03,0.998750,0.997189
